In [ ]:
%pip install -U pandas==1.5.3

Python interpreter will be restarted.
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Not uninstalling pandas at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-1cb6b41d-ef3e-4289-bb3f-622fcf7711a8
    Can't uninstall 'pandas'. No files were found to uninstall.
Python interpreter will be restarted.


In [ ]:
%run "/Users/zhouzi.hu/core_func"

In [ ]:
# Libraries
from pyspark.sql import SparkSession
import random

In [ ]:
# Preprocessing 
spark=SparkSession.builder.getOrCreate()
raw=spark.read.csv('/mnt/input data for ml projects/Imported Raw Input/raw input for training propensity 2023-07-06.csv',header=True)
raw=raw.toPandas()
raw = raw.replace('NULL', np.nan)
dropna_df=raw.dropna(axis=0)
dropna_df.head(5)

,CompanyId,DistSKUcnt,Ordercnt,average_basket_size,cohort,AOV,DistCampaigncnt(past2yrs),MonthlyAvgRev(past2yrs),Have_subscription_history?,Churn? (2 months)
0,1003161,39,18,3.50,2013,68.4961,1,58.00,0,0
1,1003266,76,68,5.91,2013,165.0554,5,200.0818,0,0
2,1010512,66,74,2.96,2012,70.3375,3,83.53,0,0
3,1010611,32,53,5.15,2013,105.2935,2,145.2181,0,0
4,1013010,48,42,3.05,2012,55.5045,1,46.7225,1,0


In [ ]:
# count total input data
total_rows = len(dropna_df)
print("Total rows:", total_rows)

Total rows: 489127


In [ ]:
# Sampling based on total input

min_length = 40000
max_length = 50000

if len(dropna_df) > max_length:
    percentage = random.uniform(min_length / len(dropna_df), max_length / len(dropna_df))
    sample_df = dropna_df.sample(frac=percentage, random_state=42)
else:
    sample_df = dropna_df.copy()

sample_df.head(5)

,CompanyId,DistSKUcnt,Ordercnt,average_basket_size,cohort,AOV,DistCampaigncnt(past2yrs),MonthlyAvgRev(past2yrs),Have_subscription_history?,Churn? (2 months)
157147,2949614,2,3,1.33,2021,33.8333,1,33.8333,0,0
233160,3536668,1,1,1.00,2023,28.90,0,28.90,0,0
359809,3569846,1,1,1.00,2023,27.20,1,27.20,0,0
297249,3547176,2,1,2.00,2023,45.00,0,45.00,0,0
426594,3642279,1,1,2.00,2023,66.30,1,66.30,0,0


In [ ]:
# Obtain a list of the column headers
col_names = sample_df.columns.tolist()
# Check if feature columns have been applied correctly
print(col_names)

['CompanyId', 'DistSKUcnt', 'Ordercnt', 'average_basket_size', 'cohort', 'AOV', 'DistCampaigncnt(past2yrs)', 'MonthlyAvgRev(past2yrs)', 'Have_subscription_history?', 'Churn? (2 months)']


In [ ]:
# Isolate target data
churn_result=sample_df['Churn? (2 months)']
# Get a Boolean Label of the Churn Result
y=np.asarray(churn_result)
# Remove columns that are not required
to_drop=['CompanyId','Churn? (2 months)']
churn_feat_space=sample_df.drop(to_drop,axis=1)
# Pull out features for future use
features=churn_feat_space.columns
# Getting Feature List for training in the future
X=churn_feat_space.values.astype(np.float64)

In [ ]:
# Standardize features by removing the mean and scaling to unit variance
scaler=StandardScaler()
X=scaler.fit_transform(X)
print("Feature space holds %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(y))
#### End of Preprocessing of Data Points

Feature space holds 45273 observations and 8 features
Unique target labels: ['0' '1']


In [ ]:
target_raw=spark.read.csv('/mnt/input data for ml projects/Imported Raw Input/Current Active Customer Data 2023-07-06.csv',header=True)
target_df=target_raw.toPandas()
target_df = target_df.replace('NULL', np.nan)
dropna_target_df=target_df.dropna(axis=0)
dropna_target_df.head(5)

,CompanyId,DistSKUcnt,Ordercnt,average_basket_size,cohort,AOV,DistCampaigncnt(past2yrs),MonthlyAvgRev(past2yrs),Have_subscription_history?
0,1000016,58,64,3.78,2013,70.622,4,69.995,0
1,1000660,58,118,3.36,2012,71.4075,9,111.8466,0
2,1000919,68,135,2.56,2012,58.9476,20,74.1764,1
3,1003717,103,208,6.02,2012,110.6712,24,265.45,1
4,1003860,126,98,2.40,2013,56.7523,5,79.7027,0


In [ ]:
total_rows = len(dropna_target_df)
print("Total rows:", total_rows)

Total rows: 456219


In [ ]:
# Preprocessing target data
target_to_drop=['CompanyId']
target_churn_feat_space=dropna_target_df.drop(target_to_drop,axis=1)
X_target=target_churn_feat_space.values.astype(np.float64)
X_target_scaled=scaler.fit_transform(X_target)
print("Feature space holds %d observations and %d features" % X_target_scaled.shape)

Feature space holds 456219 observations and 8 features


In [ ]:
# Fit the GBC model
GBC_M = GBC()
GBC_M_fit = GBC_M.fit(X, y)

# Make predictions on the scaled target data
GBC_predictions = GBC_M_fit.predict_proba(X_target_scaled)

print(GBC_predictions)


[[0.94516583 0.05483417]
 [0.97157747 0.02842253]
 [0.97635069 0.02364931]
 ...
 [0.67994527 0.32005473]
 [0.81414538 0.18585462]
 [0.74873193 0.25126807]]


In [ ]:
# Construct result table

prediction_col=pd.DataFrame(GBC_predictions)
result_table=dropna_target_df.join(prediction_col, how='right')
result_table.head(5)

,CompanyId,DistSKUcnt,Ordercnt,average_basket_size,cohort,AOV,DistCampaigncnt(past2yrs),MonthlyAvgRev(past2yrs),Have_subscription_history?,0,1
0,1000016,58,64,3.78,2013,70.622,4,69.995,0,0.945166,0.054834
1,1000660,58,118,3.36,2012,71.4075,9,111.8466,0,0.971577,0.028423
2,1000919,68,135,2.56,2012,58.9476,20,74.1764,1,0.976351,0.023649
3,1003717,103,208,6.02,2012,110.6712,24,265.45,1,0.963604,0.036396
4,1003860,126,98,2.40,2013,56.7523,5,79.7027,0,0.968511,0.031489


In [ ]:
# Rename table
new_column_names={'DistSKUcnt':'Distinct_Item_Purchased','Ordercnt':'Order_Count','average_basket_size':'Average_Basket_Size','cohort':'Cohort','DistCampaigncnt':'Distinct_Campaign_Count','MonthlyAvgRev(past2yrs)':'Month_Avg_Revenue(past2yrs)',0:'Retention_Rate_Prediction_in_2_months(GBC)',1:'Churn_Rate_Prediction_in_2_months(GBC)'}
result_table.rename(columns=new_column_names,inplace=True)
result_table.head(5)

,CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,DistCampaigncnt(past2yrs),Month_Avg_Revenue(past2yrs),Have_subscription_history?,Retention_Rate_Prediction_in_2_months(GBC),Churn_Rate_Prediction_in_2_months(GBC)
0,1000016,58,64,3.78,2013,70.622,4,69.995,0,0.945166,0.054834
1,1000660,58,118,3.36,2012,71.4075,9,111.8466,0,0.971577,0.028423
2,1000919,68,135,2.56,2012,58.9476,20,74.1764,1,0.976351,0.023649
3,1003717,103,208,6.02,2012,110.6712,24,265.45,1,0.963604,0.036396
4,1003860,126,98,2.40,2013,56.7523,5,79.7027,0,0.968511,0.031489


In [ ]:
# run BG-NBD Model

In [ ]:
%run "/Users/zhouzi.hu/Predictive Behavior-Lapsed Customers/LTV_core"

,CompanyId,OrderNumTwo,PlacedDttm,PlacedUnits,PlacedRev,PlacedOrderCnt
0,1321864,307310231,2021-11-11 06:04:11.000,2,28.05,73
1,2405215,307189673,2021-10-17 08:18:30.000,3,96.50,10
2,2074145,307141394,2021-10-07 11:48:36.000,9,191.25,9
3,1724260,307192452,2021-10-17 20:08:34.000,4,82.40,19
4,1937528,307578439,2021-12-28 19:57:50.823,2,31.50,34


Total rows: 1561579


,frequency,recency,T,monetary_value
CompanyId,,,,
1000007,6.0,85.0,91.0,110.750000
1000010,28.0,99.0,99.0,117.328929
1000011,6.0,88.0,97.0,86.558333
1000013,10.0,86.0,89.0,75.970000
1000014,8.0,85.0,88.0,55.400000


,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
CompanyId,,,,,,,
1000007,5.0,56.0,83.0,94.900000,1.0,190.00,8.0
1000010,25.0,89.0,91.0,116.900400,2.0,146.15,8.0
1000011,6.0,88.0,89.0,86.558333,0.0,0.00,8.0
1000013,9.0,73.0,81.0,78.544444,1.0,52.80,8.0
1000014,7.0,68.0,80.0,49.257143,1.0,98.40,8.0


Out[31]: <lifetimes.BetaGeoFitter: fitted with 364638 subjects, a: 0.00, alpha: 7.20, b: 0.00, r: 0.34>

...

Out[34]: 242501

Out[35]: CompanyId
1000007    187.221462
1000010    128.586589
1000011    146.342219
1000013    100.648602
1000014     79.893966
dtype: float64

Out[36]: CompanyId
1000007     97.569026
1000010    276.876845
1000011     71.873460
1000013     87.307550
1000014     56.490115
Name: clv, dtype: float64

           trans_pred_in_2months_BGF  alive_prob_BGF  \
CompanyId                                              
1000007                     0.555526             1.0   
1000010                     2.295300             1.0   
1000011                     0.523537             1.0   
1000013                     0.924682             1.0   
1000014                     0.753715             1.0   

           Expected_Rev_in_2months_BGF  Expected_Avg_Rev_for_returningCus_GG  \
CompanyId                                                                      
1000007                      61.524560                            187.221462   
1000010                     269.305058                            128.586589   
1000011                      45.316488                            146.342219   
1000013                      70.248106                            100.648602   
1000014                      41.755788                             79.893966   

           pred_CLV_in2months_GG  
CompanyId  

,trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG
CompanyId,,,,,
1000007,0.555526,1.0,61.524560,187.221462,97.569026
1000010,2.295300,1.0,269.305058,128.586589,276.876845
1000011,0.523537,1.0,45.316488,146.342219,71.873460
1000013,0.924682,1.0,70.248106,100.648602,87.307550
1000014,0.753715,1.0,41.755788,79.893966,56.490115


In [ ]:
merged_table= pd.merge(result_table, result_table2, left_on='CompanyId', right_index=True)
merged_table=merged_table.reset_index(drop=True)
merged_table.head(5)

,CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,DistCampaigncnt(past2yrs),Month_Avg_Revenue(past2yrs),Have_subscription_history?,Retention_Rate_Prediction_in_2_months(GBC),Churn_Rate_Prediction_in_2_months(GBC),trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG
0,1000016,58,64,3.78,2013,70.622,4,69.995,0,0.945166,0.054834,0.924682,1.0,59.766834,85.636752,74.285533
1,1000660,58,118,3.36,2012,71.4075,9,111.8466,0,0.971577,0.028423,1.378526,1.0,130.692404,110.782806,143.264630
2,1000919,68,135,2.56,2012,58.9476,20,74.1764,1,0.976351,0.023649,1.695330,1.0,98.752957,66.394358,105.593434
3,1003717,103,208,6.02,2012,110.6712,24,265.45,1,0.963604,0.036396,3.717980,1.0,464.581834,131.989092,460.359037
4,1003860,126,98,2.40,2013,56.7523,5,79.7027,0,0.968511,0.031489,1.611993,1.0,117.170653,83.459759,126.209409


In [ ]:
merged_table.columns

Out[40]: Index(['CompanyId', 'Distinct_Item_Purchased', 'Order_Count',
       'Average_Basket_Size', 'Cohort', 'AOV', 'DistCampaigncnt(past2yrs)',
       'Month_Avg_Revenue(past2yrs)', 'Have_subscription_history?',
       'Retention_Rate_Prediction_in_2_months(GBC)',
       'Churn_Rate_Prediction_in_2_months(GBC)', 'trans_pred_in_2months_BGF',
       'alive_prob_BGF', 'Expected_Rev_in_2months_BGF',
       'Expected_Avg_Rev_for_returningCus_GG', 'pred_CLV_in2months_GG'],
      dtype='object')

In [ ]:
new_column_names={'DistCampaigncnt(past2yrs)':'distCampaigncnt_past2yrs','Month_Avg_Revenue(past2yrs)':'monthly_rev_past2yrs','Have_subscription_history?':'have_subscription_history','Retention_Rate_Prediction_in_2_months(GBC)':'retention_pred_2month_GBC','Churn_Rate_Prediction_in_2_months(GBC)':'churn_pred_2month_GBC'}
merged_table.rename(columns=new_column_names,inplace=True)
merged_table.head(5)

,CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,distCampaigncnt_past2yrs,monthly_rev_past2yrs,have_subscription_history,retention_pred_2month_GBC,churn_pred_2month_GBC,trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG
0,1000016,58,64,3.78,2013,70.622,4,69.995,0,0.945166,0.054834,0.924682,1.0,59.766834,85.636752,74.285533
1,1000660,58,118,3.36,2012,71.4075,9,111.8466,0,0.971577,0.028423,1.378526,1.0,130.692404,110.782806,143.264630
2,1000919,68,135,2.56,2012,58.9476,20,74.1764,1,0.976351,0.023649,1.695330,1.0,98.752957,66.394358,105.593434
3,1003717,103,208,6.02,2012,110.6712,24,265.45,1,0.963604,0.036396,3.717980,1.0,464.581834,131.989092,460.359037
4,1003860,126,98,2.40,2013,56.7523,5,79.7027,0,0.968511,0.031489,1.611993,1.0,117.170653,83.459759,126.209409


In [ ]:
# ntile


num_tiles = 10  
merged_table['Cohort_Retention_Group'] = pd.qcut(merged_table['churn_pred_2month_GBC'], num_tiles, labels=False, duplicates='drop')

# Sort the DataFrame by the column in descending order
merged_table = merged_table.sort_values('retention_pred_2month_GBC', ascending=False)

# Print the resulting DataFrame
merged_table.head(5)


,CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,distCampaigncnt_past2yrs,monthly_rev_past2yrs,have_subscription_history,retention_pred_2month_GBC,churn_pred_2month_GBC,trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG,Cohort_Retention_Group
259481,3562550,33,9,8.89,2023,217.1055,2,390.79,0,0.997040,0.002960,1.823755,1.0,425.664436,457.626048,782.940590,0
303461,3606282,13,3,4.67,2023,176.5166,1,264.775,0,0.996490,0.003510,0.571920,1.0,64.970082,NaN,NaN,0
341971,3637309,39,5,8.00,2023,242.94,3,416.1466,0,0.996458,0.003542,1.244193,1.0,339.925857,NaN,NaN,0
175590,3520231,9,3,5.33,2023,176.7333,1,199.7066,0,0.996130,0.003870,0.395615,1.0,97.902818,NaN,NaN,0
176860,3520470,28,4,9.25,2023,239.11,2,318.8133,0,0.995933,0.004067,0.984750,1.0,244.116308,1350.926385,1247.984948,0


In [ ]:
# define a propensity score

def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

def calculate_propensity_score(predicted_retention_rate, predicted_transactions,alive_probability,pred_CLV):
    retention_weight = 0.6
    transaction_weight = 0.3
    CLV_weight = 0.1

    # Find the minimum and maximum values for normalization
    min_retention_rate = min(predicted_retention_rate)
    max_retention_rate = max(predicted_retention_rate)
    min_transactions = min(predicted_transactions)
    max_transactions = max(predicted_transactions)

    # Calculate the average of pred_CLV, ignoring NaN values
    avg_CLV = np.nanmean(pred_CLV)

    # Replace NaN values in pred_CLV with the average CLV
    pred_CLV = np.where(np.isnan(pred_CLV), avg_CLV, pred_CLV)

    # Find the minimum and maximum values for normalized CLV (including the average CLV)
    min_CLV = min(min(pred_CLV), avg_CLV)
    max_CLV = max(max(pred_CLV), avg_CLV)

    # Calculate normalized values
    normalized_retention_rate = [normalize(value, min_retention_rate, max_retention_rate) for value in predicted_retention_rate]
    normalized_transactions = [normalize(value, min_transactions, max_transactions) for value in predicted_transactions]
    normalized_CLV = [normalize(value, min_CLV, max_CLV) for value in pred_CLV]

    # Calculate the propensity score
    propensity_score = [
        (retention_weight * retention) + (transaction_weight * transactions * alive_prob) + (CLV_weight * CLV * alive_prob)
        for retention, transactions, alive_prob, CLV in zip(normalized_retention_rate, normalized_transactions, alive_probability, normalized_CLV)
    ]

    return propensity_score

In [ ]:
# calculate propensity score

merged_table['propensity_score']=calculate_propensity_score(merged_table['retention_pred_2month_GBC'],merged_table['trans_pred_in_2months_BGF'],merged_table['alive_prob_BGF'],merged_table['pred_CLV_in2months_GG'])

merged_table.head(5)

,CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,distCampaigncnt_past2yrs,monthly_rev_past2yrs,have_subscription_history,retention_pred_2month_GBC,churn_pred_2month_GBC,trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG,Cohort_Retention_Group,propensity_score
259481,3562550,33,9,8.89,2023,217.1055,2,390.79,0,0.997040,0.002960,1.823755,1.0,425.664436,457.626048,782.940590,0,0.702387
303461,3606282,13,3,4.67,2023,176.5166,1,264.775,0,0.996490,0.003510,0.571920,1.0,64.970082,NaN,NaN,0,0.627005
341971,3637309,39,5,8.00,2023,242.94,3,416.1466,0,0.996458,0.003542,1.244193,1.0,339.925857,NaN,NaN,0,0.657550
175590,3520231,9,3,5.33,2023,176.7333,1,199.7066,0,0.996130,0.003870,0.395615,1.0,97.902818,NaN,NaN,0,0.618699
176860,3520470,28,4,9.25,2023,239.11,2,318.8133,0,0.995933,0.004067,0.984750,1.0,244.116308,1350.926385,1247.984948,0,0.675699


In [ ]:
# define propensity group


# Calculate the percentiles
percentiles = np.percentile(merged_table['propensity_score'], [33.33, 66.67])

# Create a new column to hold the group labels
merged_table['propensity_group'] = pd.cut(merged_table['propensity_score'], bins=[merged_table['propensity_score'].min()] + list(percentiles) + [merged_table['propensity_score'].max()], labels=['Low', 'Medium', 'High'])

merged_table.head(5)

,CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,distCampaigncnt_past2yrs,monthly_rev_past2yrs,have_subscription_history,retention_pred_2month_GBC,churn_pred_2month_GBC,trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG,Cohort_Retention_Group,propensity_score,propensity_group
259481,3562550,33,9,8.89,2023,217.1055,2,390.79,0,0.997040,0.002960,1.823755,1.0,425.664436,457.626048,782.940590,0,0.702387,High
303461,3606282,13,3,4.67,2023,176.5166,1,264.775,0,0.996490,0.003510,0.571920,1.0,64.970082,NaN,NaN,0,0.627005,High
341971,3637309,39,5,8.00,2023,242.94,3,416.1466,0,0.996458,0.003542,1.244193,1.0,339.925857,NaN,NaN,0,0.657550,High
175590,3520231,9,3,5.33,2023,176.7333,1,199.7066,0,0.996130,0.003870,0.395615,1.0,97.902818,NaN,NaN,0,0.618699,High
176860,3520470,28,4,9.25,2023,239.11,2,318.8133,0,0.995933,0.004067,0.984750,1.0,244.116308,1350.926385,1247.984948,0,0.675699,High


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession


# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert the DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(merged_table)

# Register the DataFrame as a temporary table
spark_df.createOrReplaceTempView("temp_table")

# Define the database and view names
database_name = "dtc"
view_name = "propensity_na"


# Drop the old view if it exists
spark.sql(f"DROP TABLE IF EXISTS {database_name}.{view_name}")

# Create a persistent table from the temporary table
spark.sql(f"CREATE TABLE {database_name}.{view_name} USING DELTA AS SELECT * FROM temp_table")



Out[46]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [ ]:
%sql

----table check----
select *
from dtc.propensity_na a 
limit 1000


CompanyId,Distinct_Item_Purchased,Order_Count,Average_Basket_Size,Cohort,AOV,distCampaigncnt_past2yrs,monthly_rev_past2yrs,have_subscription_history,retention_pred_2month_GBC,churn_pred_2month_GBC,trans_pred_in_2months_BGF,alive_prob_BGF,Expected_Rev_in_2months_BGF,Expected_Avg_Rev_for_returningCus_GG,pred_CLV_in2months_GG,Cohort_Retention_Group,propensity_score,propensity_group
2698404,10,5,4.20,2021,110.772,1,146.26,0,0.6783622531305249,0.32163774686947516,0.31525854612283194,1.0,37.357086853735176,646.0076680598083,191.05398442146847,8,0.36276433703487426,Low
2885549,16,5,3.60,2021,104.08,0,146.00,0,0.6783622531305249,0.32163774686947516,0.5140946043313819,0.9999999999999998,87.70453949893373,null,null,8,0.36950775008917347,Low
2912024,11,5,3.80,2021,107.174,0,145.1833,0,0.6783622531305249,0.32163774686947516,0.23932570786044965,0.9999999999999998,37.71533830172825,null,null,8,0.35701319529362274,Low
3061704,12,5,3.60,2021,88.88,0,148.1333,0,0.6783622531305249,0.32163774686947516,0.31875383302123367,0.9999999999999998,29.240351615814497,500.21188299167113,149.57567738293608,8,0.3618217080802296,Low
2629114,11,5,4.80,2021,94.368,0,148.50,0,0.6783622531305249,0.32163774686947516,0.16454238129815055,0.999999999879879,24.187730047922678,null,null,8,0.35361257614123087,Low
2668214,13,5,3.80,2021,96.63,0,151.2166,0,0.6783622531305249,0.32163774686947516,0.4154920196293344,1.0,53.854690610955565,706.5849179025416,275.4093040001669,8,0.36956253611707063,Low
2655872,13,5,3.80,2021,102.46,1,143.9366,0,0.6783622531305249,0.32163774686947516,0.3138874299020345,0.9999999999999998,36.253998153684975,null,null,8,0.3604037374316328,Low
1350446,4,3,1.33,2014,26.9833,0,25.60,0,0.6783345048858587,0.3216654951141414,0.4247092214254591,0.9999999996564193,11.89185819582704,null,null,8,0.36542090175444325,Low
1484339,3,3,1.33,2015,24.1333,1,22.40,0,0.6783345048858587,0.3216654951141414,0.13291564815393997,1.0,0.0,null,null,8,0.3521521845339074,Low
1496197,7,3,5.67,2015,34.80,1,25.50,0,0.6783345048858587,0.3216654951141414,0.24190464361062147,1.0,0.0,null,null,8,0.35710823666839486,Low
